# Лабораторна робота №1. Симплекс-метод розв’язання задачі лінійного програмування

Виконав - Огоновський Олександр

**Мета**:  навчитися розв’язувати задачу лінійного програмування (ЗЛП) симплекс-методом.


# Графічний метод

Результат з Пз 1:

$
\min F = 0,\ \max F = 15
$

Мінімум досягається на всьому відрізку між $( A\left(\frac{10}{3},\frac{2}{3}\right) )$ і $( B\left(\frac{25}{2},\frac{5}{2}\right) )$ (ребро $( x_1-5x_2=0 )$ всередині багатокутника).  
Максимум досягається в точці $( D(15,0) )$.

## (Завдання 1) Розв’язання з використанням пакета SciPy

In [4]:
import numpy as np
from scipy.optimize import linprog

# Коефіцієнти цільової функції F = x1 - 5*x2
c_min = [1, -5]  # для мінімізації
c_max = [-1, 5]  # для максимізації (змінюємо знак)

# Матриця обмежень A_ub * x <= b_ub
# Перетворюємо обмеження:
# 1) x1 - 5*x2 >= 0 -> -x1 + 5*x2 <= 0
# 2) x1 + x2 <= 15
# 3) x1 + x2 >= 4 -> -x1 - x2 <= -4
# 4) x1 >= 0, x2 >= 0 задамо через bounds

A_ub = [
    [-1, 5],   # -x1 + 5*x2 <= 0
    [1, 1],    # x1 + x2 <= 15
    [-1, -1]   # -x1 - x2 <= -4
]

b_ub = [0, 15, -4]

# Границі на змінні
bounds = [(0, None), (0, None)]

# Розв'язання задачі мінімізації
res_min = linprog(c_min, A_ub=A_ub, b_ub=b_ub, bounds=bounds, method='highs')
print("=== Мінімізація ===")
print(f"Статус: {res_min.message}")
print(f"Мінімальне значення F: {res_min.fun:.4f}")
print(f"Точка мінімуму: x1 = {res_min.x[0]:.4f}, x2 = {res_min.x[1]:.4f}")
print()

# Розв'язання задачі максимізації
res_max = linprog(c_max, A_ub=A_ub, b_ub=b_ub, bounds=bounds, method='highs')
print("=== Максимізація ===")
print(f"Статус: {res_max.message}")
print(f"Максимальне значення F: {-res_max.fun:.4f}")  # Мінус, бо мінімізували -F
print(f"Точка максимуму: x1 = {res_max.x[0]:.4f}, x2 = {res_max.x[1]:.4f}")
print()

# Перевірка, що відрізок між A і B дає мінімум F=0
print("=== Перевірка відрізка мінімуму ===")
# Точки з аналітичного розв'язку
A = [10/3, 2/3]
B = [25/2, 5/2]
mid_point = [(A[0] + B[0])/2, (A[1] + B[1])/2]

print(f"Точка A: F = {A[0] - 5*A[1]:.6f}")
print(f"Точка B: F = {B[0] - 5*B[1]:.6f}")
print(f"Середина AB: F = {mid_point[0] - 5*mid_point[1]:.6f}")

=== Мінімізація ===
Статус: Optimization terminated successfully. (HiGHS Status 7: Optimal)
Мінімальне значення F: -0.0000
Точка мінімуму: x1 = 3.3333, x2 = 0.6667

=== Максимізація ===
Статус: Optimization terminated successfully. (HiGHS Status 7: Optimal)
Максимальне значення F: 15.0000
Точка максимуму: x1 = 15.0000, x2 = 0.0000

=== Перевірка відрізка мінімуму ===
Точка A: F = 0.000000
Точка B: F = 0.000000
Середина AB: F = 0.000000


## (Завдання 3) Розв’язання з PuLP

In [6]:
from pulp import LpProblem, LpVariable, LpMinimize, LpMaximize, LpStatus, value

# Створюємо задачу для мінімізації
prob_min = LpProblem("Minimization_Problem", LpMinimize)

# Оголошуємо змінні
x1 = LpVariable('x1', lowBound=0)  # x1 >= 0
x2 = LpVariable('x2', lowBound=0)  # x2 >= 0

# Цільова функція для мінімізації
prob_min += x1 - 5*x2, "Objective_Function"

# Обмеження
prob_min += x1 - 5*x2 >= 0, "Constraint1"
prob_min += x1 + x2 <= 15, "Constraint2"
prob_min += x1 + x2 >= 4, "Constraint3"

# Розв'язуємо задачу мінімізації
prob_min.solve()

print("=== Мінімізація ===")
print(f"Статус: {LpStatus[prob_min.status]}")
print(f"Мінімальне значення F: {value(prob_min.objective):.4f}")
print(f"Точка мінімуму: x1 = {value(x1):.4f}, x2 = {value(x2):.4f}")
print()

# Створюємо задачу для максимізації
prob_max = LpProblem("Maximization_Problem", LpMaximize)

# Оголошуємо нові змінні (можна використати ті самі імена, але в іншому об'єкті)
x1_max = LpVariable('x1_max', lowBound=0)
x2_max = LpVariable('x2_max', lowBound=0)

# Цільова функція для максимізації
prob_max += x1_max - 5*x2_max, "Objective_Function"

# Обмеження (ті самі)
prob_max += x1_max - 5*x2_max >= 0, "Constraint1"
prob_max += x1_max + x2_max <= 15, "Constraint2"
prob_max += x1_max + x2_max >= 4, "Constraint3"

# Розв'язуємо задачу максимізації
prob_max.solve()

print("=== Максимізація ===")
print(f"Статус: {LpStatus[prob_max.status]}")
print(f"Максимальне значення F: {value(prob_max.objective):.4f}")
print(f"Точка максимуму: x1 = {value(x1_max):.4f}, x2 = {value(x2_max):.4f}")
print()


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/fobosogon/lb5/.venv/lib/python3.12/site-packages/pulp/apis/../solverdir/cbc/linux/i64/cbc /tmp/47b666eaa57d40238e0104cb1126ab7a-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /tmp/47b666eaa57d40238e0104cb1126ab7a-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 8 COLUMNS
At line 17 RHS
At line 21 BOUNDS
At line 22 ENDATA
Problem MODEL has 3 rows, 2 columns and 6 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 3 (0) rows, 2 (0) columns and 6 (0) elements
0  Obj 0 Primal inf 3.9999999 (1) Dual inf 2.2360679 (1)
0  Obj 0 Primal inf 3.9999999 (1) Dual inf 1.4472136e+10 (2)
2  Obj 0
Optimal - objective value 4.4408921e-16
Optimal objective 4.440892099e-16 - 2 iterations time 0.002
Option for printingOptions changed from normal to all
Total time (CPU seconds):       0.00   (Wallclock seconds):

# Симплекс-метод

Результат в ПЗ №2:

 $ \boxed{Z_{\max}=160} $

$(x_1 = 2)$ (DC)  

$(x_3 = 2)$ (Coder)  

$(x_2 = 0)$ (MUX) 



## (Завдання 2) Розв’язання з використанням пакета SciPy

In [1]:
import numpy as np
from scipy.optimize import linprog

# Коефіцієнти цільової функції (мінус, бо linprog мінімізує, а нам треба максимізувати)
c = [-40, -45, -40]  # Максимізуємо Z = 40*x1 + 45*x2 + 40*x3

# Коефіцієнти обмежень (ліві частини нерівностей)
A = [
    [2, 3, 1],    # K1: 2x1 + 3x2 + x3 <= 6
    [4, 5, 5],    # K2: 4x1 + 5x2 + 5x3 <= 18
    [10, 8, 8]    # K3: 10x1 + 8x2 + 8x3 <= 120
]

# Праві частини обмежень
b = [6, 18, 120]

# Діапазони змінних (x1, x2, x3 >= 0)
x_bounds = [(0, None), (0, None), (0, None)]

# Розв'язання задачі лінійного програмування
result = linprog(c, A_ub=A, b_ub=b, bounds=x_bounds, method='highs')

# Виведення результатів
print("Статус розв'язку:", result.message)
print("Оптимальні значення змінних:")
print(f"x1 (Дешифратори) = {result.x[0]:.2f}")
print(f"x2 (Мультиплексори) = {result.x[1]:.2f}")
print(f"x3 (Шифратори) = {result.x[2]:.2f}")
print(f"\nМаксимальна вартість: {-result.fun:.2f}")

Статус розв'язку: Optimization terminated successfully. (HiGHS Status 7: Optimal)
Оптимальні значення змінних:
x1 (Дешифратори) = 2.00
x2 (Мультиплексори) = 0.00
x3 (Шифратори) = 2.00

Максимальна вартість: 160.00


## (Завдання 4) Розв’язання з PuLP

In [3]:
from pulp import LpMaximize, LpProblem, LpVariable, lpSum, LpStatus, value

# Створюємо задачу лінійного програмування
problem = LpProblem("Maximize_Profit", LpMaximize)

# Оголошуємо змінні (можна додати цілочисельність)
x1 = LpVariable("DC", lowBound=0, cat='Integer')  # Дешифратори
x2 = LpVariable("MUX", lowBound=0, cat='Integer')  # Мультиплексори
x3 = LpVariable("Coder", lowBound=0, cat='Integer')  # Шифратори

# Цільова функція
problem += 40*x1 + 45*x2 + 40*x3, "Total_Value"

# Обмеження на кількість елементів K1 (АБО-НЕ)
problem += 2*x1 + 3*x2 + 1*x3 <= 6, "K1_elements"

# Обмеження на кількість елементів K2 (І-НЕ)
problem += 4*x1 + 5*x2 + 5*x3 <= 18, "K2_elements"

# Обмеження на кількість елементів K3 (XOR, І, 1)
problem += 10*x1 + 8*x2 + 8*x3 <= 120, "K3_elements"

# Розв'язуємо задачу
problem.solve()

# Виводимо результати
print("="*50)
print("Розв'язок задачі виробництва вузлів ЕОМ")
print("="*50)
print(f"Статус розв'язку: {LpStatus[problem.status]}")
print(f"Оптимальне значення цільової функції: {value(problem.objective):.0f}")
print()
print("Оптимальний план виробництва:")
print(f"Дешифратори (DC): {value(x1):.0f} од.")
print(f"Мультиплексори (MUX): {value(x2):.0f} од.")
print(f"Шифратори (Coder): {value(x3):.0f} од.")
print()

==================================================Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/fobosogon/lb5/.venv/lib/python3.12/site-packages/pulp/apis/../solverdir/cbc/linux/i64/cbc /tmp/781028824bd241eeb433360f637e453c-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /tmp/781028824bd241eeb433360f637e453c-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 8 COLUMNS
At line 27 RHS
At line 31 BOUNDS
At line 35 ENDATA
Problem MODEL has 3 rows, 3 columns and 9 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 160 - 0.00 seconds
Cgl0004I processed model has 2 rows, 3 columns (3 integer (0 of which binary)) and 6 elements
Cutoff increment increased from 1e-05 to 4.9999
Cbc0012I Integer solution of -160 found by DiveCoefficient after 0 iterations and 0 nodes (0.00 seconds)
Cbc0001I Search completed - best objective 